In [2]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
import numpy as np
import matplotlib.pyplot as plt
import os

gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_visible_devices(gpus[1], 'GPU')
tf.config.experimental.set_memory_growth(gpus[1], True)

AUTOTUNE = tf.data.AUTOTUNE

from shared_funcs import multi_label_binary_encode_tensor, multi_label_binary_decode_tensor, get_waveform, split_into_windows, split_into_sequences 

2024-12-04 20:10:37.970429: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-04 20:10:37.984849: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-04 20:10:37.989216: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-04 20:10:38.001105: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-04 20:10:39.948874: W tensorflow/compiler/tf2

In [3]:
ALL_LABELS = tf.constant(['Rctrl', 'p', 'esc', 'g', 'slash', 'down', '7', 'equal', 'w', 'a', 'dash', 'caps', 'l', 'd', 'backspace', 'bracketclose', 'z', '1', 'end', 'Rshift', 'comma', 'c', 'tab', 'b', 'j', 'right', 'Lctrl', 'n', 't', 'f', 'm', 'o', 'apostrophe', 'y', '8', 'space', 'backslash', 's', '9', 'i', 'r', 'bracketopen', 'semicolon', 'q', '5', 'k', '3', 'x', '4', '6', '2', 'Lshift', 'left', 'backtick', 'enter', 'fullstop', 'e', '0', 'h', 'v', 'up', 'u', 'delete'], dtype=tf.string)


### Load Datasets

In [4]:
# Load MKA Dataset
# mka_path = os.path.dirname(os.getcwd()) + "/data-manipulation/mka_dataset"
# mka_dataset = tf.data.Dataset.load(mka_path)

# Load supplementary dataset
supp_path = os.path.dirname(os.getcwd()) + "/data-manipulation/supp_dataset"
supp_dataset = tf.data.Dataset.load(supp_path)

# mka_dataset.cache()
supp_dataset.cache()
# mka_dataset = mka_dataset.prefetch(tf.data.AUTOTUNE)
supp_dataset = supp_dataset.prefetch(tf.data.AUTOTUNE)

# print(f"MKA Dataset {mka_dataset.element_spec}\nLength: {tf.data.experimental.cardinality(mka_dataset).numpy()}")
print(f"Supp Dataset {supp_dataset.element_spec}\nLength: {tf.data.experimental.cardinality(supp_dataset).numpy()}")

spectrogram_dataset = supp_dataset

# Add channel dimension for conv network
spectrogram_dataset = spectrogram_dataset.map(
    lambda spectrogram, label: (tf.expand_dims(spectrogram, axis=-1), label),
    num_parallel_calls=tf.data.AUTOTUNE
)

Supp Dataset (TensorSpec(shape=(39, 1103), dtype=tf.float32, name=None), TensorSpec(shape=(39, 63), dtype=tf.int32, name=None))
Length: 16


### Prepare Dataset for Training

In [5]:
dataset_size = sum(1 for _ in spectrogram_dataset)
train_size = int(0.8 * dataset_size)
val_size = int(0.1 * dataset_size)

# Split into train val and test datasets
train_dataset = spectrogram_dataset.take(train_size)
remaining_dataset = spectrogram_dataset.skip(train_size)
val_dataset = remaining_dataset.take(val_size)
test_dataset = remaining_dataset.skip(val_size)

2024-12-04 20:10:43.518896: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


### Calculate Class Weights
Since the dataset may be imbalanced with different classes, especially with the no key pressed class, we must calculate the class weights so the model is not biased towards the majority class

In [6]:
def calculate_class_weights(labels):
    print(labels.shape)
    num_samples, num_classes = labels.shape
    class_counts = np.sum(labels, axis=0)
    total_counts = np.sum(class_counts)
    
    class_weights = {i: total_counts / (num_classes * class_counts[i]) for i in range(num_classes)}
    return class_weights

# Example usage
all_labels = np.concatenate([label.numpy() for _, label in train_dataset])  # Gather all labels
class_weights = calculate_class_weights(all_labels)
print("Class Weights:", class_weights)

(468, 63)
Class Weights: {0: inf, 1: inf, 2: inf, 3: inf, 4: inf, 5: inf, 6: inf, 7: inf, 8: 0.3148148148148148, 9: 0.1574074074074074, 10: inf, 11: inf, 12: 0.1574074074074074, 13: 0.20987654320987653, 14: inf, 15: inf, 16: 0.6296296296296297, 17: inf, 18: inf, 19: inf, 20: inf, 21: inf, 22: inf, 23: inf, 24: 0.20987654320987653, 25: inf, 26: inf, 27: 1.8888888888888888, 28: 0.6296296296296297, 29: 0.6296296296296297, 30: inf, 31: 0.41975308641975306, 32: inf, 33: inf, 34: inf, 35: 0.09444444444444444, 36: inf, 37: 0.6296296296296297, 38: inf, 39: 0.6296296296296297, 40: 0.37777777777777777, 41: inf, 42: inf, 43: inf, 44: inf, 45: 0.1574074074074074, 46: inf, 47: inf, 48: inf, 49: inf, 50: inf, 51: inf, 52: inf, 53: inf, 54: inf, 55: inf, 56: 0.2222222222222222, 57: inf, 58: 0.37777777777777777, 59: inf, 60: inf, 61: inf, 62: inf}


2024-12-04 20:10:43.556298: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
/tmp/ipykernel_1728213/1588371172.py:7: RuntimeWarning: divide by zero encountered in scalar divide
  class_weights = {i: total_counts / (num_classes * class_counts[i]) for i in range(num_classes)}


### Batch and Prefetch datasets

In [7]:
# Batch and prefetch
train_dataset = train_dataset.batch(32).prefetch(buffer_size=AUTOTUNE)
val_dataset = val_dataset.batch(32).prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.batch(32).prefetch(buffer_size=AUTOTUNE)

# Model

In [8]:
print(spectrogram_dataset.element_spec)
model = keras.Sequential([
    layers.Input(shape=(39, 1103, 1)),
    layers.Conv2D(32, kernel_size=3, activation='relu', padding='same'),
    layers.Conv2D(128, kernel_size=3, activation='relu', padding='same'),
    layers.MaxPooling2D(pool_size=(1, 2)),
    layers.Dropout(0.3),

    layers.Reshape((39, -1)),
    layers.TimeDistributed(layers.Dense(128, activation='relu')),
    layers.TimeDistributed(layers.Dense(63, activation="sigmoid"))
])

model.summary()

(TensorSpec(shape=(39, 1103, 1), dtype=tf.float32, name=None), TensorSpec(shape=(39, 63), dtype=tf.int32, name=None))


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 39, 1103, 32)   │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 39, 1103, 128)  │        36,992 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 39, 551, 128)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 39, 551, 128)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape (Reshape)               │ (None, 39, 70528)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed                │ (None, 39, 128)        │     9,027,712 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_1              │ (None, 39, 63)         │         8,127 │
│ (TimeDistributed)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9,073,151 (34.61 MB)

 Trainable params: 9,073,151 (34.61 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.005),
    loss='binary_crossentropy',
    metrics=['accuracy'] # , keras.metrics.Precision()
)

model.fit(
    train_dataset,
    epochs=15,  # Adjust as needed
    validation_data=val_dataset,
    class_weight=class_weights
)

TypeError: TensorFlowTrainer.fit() got an unexpected keyword argument 'class_weights'

## Evaluate Model

In [ ]:
loss, accuracy = model.evaluate(test_dataset)
print(f"Loss: {loss}, Accuracy: {accuracy}")


# predictions = model.predict(new_data)

# threshold = 0.5
# binary_predictions = (predictions > threshold).astype(int)

# # Output predictions
# print("Raw Predictions:", predictions)
# print("Binary Predictions:", binary_predictions)

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.0855 - loss: 0.0787
Loss: 0.07874199002981186, Accuracy: 0.08547008782625198


## Optimize Model